# График кол-во страниц Википедии, запрещённых в России по годам \ Graph of the number of Wikipedia pages banned in Russia by year


### Источник данных \ Source

[Страницы Википедии, запрещённые в России \ Wikipedia pages banned in Russia](https://ru.wikipedia.org/wiki/%D0%92%D0%B8%D0%BA%D0%B8%D0%BF%D0%B5%D0%B4%D0%B8%D1%8F:%D0%A1%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B_%D0%92%D0%B8%D0%BA%D0%B8%D0%BF%D0%B5%D0%B4%D0%B8%D0%B8,_%D0%B7%D0%B0%D0%BF%D1%80%D0%B5%D1%89%D1%91%D0%BD%D0%BD%D1%8B%D0%B5_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8)

###  Описание данных \ Data Description
* 'Статья' — Название статьи \ Article title
* 'Дата основания' — Дата вынесения информации о блокировке \ Date when the block was issued 
* 'Дата внесения' — Дата вынесения информации в реестр \ Date when the block was entered into the registry
* 'Дата исключения' — Дата исключения из реестра \ Date of exclusion from the register
* 'Кто' - Ведомство внесшее в реестр \ Office that entered the register
* 'Год внесения' — Год вынесения информации в реестр \ Year when the block was entered into the registry
* 'Год исключения' — Год исключения из реестра \ Year of exclusion from the register

#####  Библиотеки и функции

In [5]:
import numpy as np
import pandas as pd
import warnings
import seaborn as sns
#import matplotlib.pyplot as plt
from termcolor import colored
from plotly import graph_objects as go

from plotly.colors import n_colors
import plotly.express as px
import plotly as plt

import json

#Импортируем все необходимые библиотеки


#import datetime
#import itertools
#import warnings

#import matplotlib.pyplot as plt
#import seaborn as sns
#import re
#import requests


user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}


In [6]:
def diff(first, second):
    return [item for item in first if item not in second]

In [7]:
years_lst = [i for i in range(2012,2023)]

##### Загрузка данных

In [8]:
#загрузим
dt = pd.read_csv('wiki_re.csv', sep=',',decimal='.')

In [23]:
dt.head()

,Статья,Дата основания,Дата внесения,Дата исключения,Кто,Год внесения,Год исключения
0,Динамит,03.06.2015,31.07.2015,NaN,Минюст,2015.0,NaN
1,2022 Russian invasion of Ukraine,17.05.2022,18.05.2022,NaN,Генпрокуратура,2022.0,NaN
2,Rashism,17.05.2022,18.05.2022,NaN,Генпрокуратура,2022.0,NaN
3,Битва за Киев (2022),04.04.2022,04.04.2022,NaN,Генпрокуратура,2022.0,NaN
4,Военные преступления в период вторжения России...,04.04.2022,04.04.2022,NaN,Генпрокуратура,2022.0,NaN


##### Обработка данных

In [9]:
dt = dt[dt['Кто'] != 'я']

In [10]:
dt['Кто'].unique()

array(['Минюст', 'Генпрокуратура', 'ФСКН', 'МВД', 'Суд',
       'Роспотребнадзор', 'Роскомнадзор'], dtype=object)

###### Таблица с блокировками

In [11]:
ban_year = dt.pivot_table(index=['Год внесения','Кто'],values='Статья', aggfunc='count').reset_index()
dl = diff(years_lst, ban_year['Год внесения'].unique())
dl
dl2 = pd.DataFrame()
dl2['Год внесения'] = dl
ban_year = ban_year.append(dl2)
ban_year['Год внесения'] = ban_year['Год внесения'].astype("int")

ban_year = ban_year.sort_values('Год внесения').fillna(0)
ban_year.rename(columns = {'Год внесения':'Год', 'Кто':'Ведомство'}, inplace = True)
ban_year.head()

,Год,Ведомство,Статья
0,2012,Роспотребнадзор,2.0
1,2012,ФСКН,8.0
2,2013,Роспотребнадзор,7.0
3,2013,ФСКН,7.0
4,2014,Роспотребнадзор,2.0


In [12]:
#ban_year.pivot_table(index='Год',values='Статья', aggfunc='sum').reset_index()

###### Таблица с отменами

In [13]:
cancellation_year = dt.pivot_table(index=['Год исключения','Кто'],values='Статья', aggfunc='count').reset_index()
dl = diff(years_lst, cancellation_year['Год исключения'].unique())
dl
dl2 = pd.DataFrame()
dl2['Год исключения'] = dl
cancellation_year = cancellation_year.append(dl2)
cancellation_year['Год исключения'] = cancellation_year['Год исключения'].astype("int")
cancellation_year = cancellation_year.sort_values('Год исключения').fillna(0)
cancellation_year.rename(columns = {'Год исключения':'Год', 'Кто':'Ведомство'}, inplace = True)
cancellation_year['Статья'] = cancellation_year['Статья']* -1
cancellation_year.head()

,Год,Ведомство,Статья
0,2012,0,-0.0
0,2013,Роспотребнадзор,-6.0
1,2013,ФСКН,-14.0
2,2014,Роспотребнадзор,-1.0
3,2014,ФСКН,-3.0


In [14]:
#cancellation_year.pivot_table(index='Год',values='Статья', aggfunc='sum').reset_index()

###### Объединили

In [15]:
dt_year= ban_year.append(cancellation_year)

In [16]:
dt_year.head()

,Год,Ведомство,Статья
0,2012,Роспотребнадзор,2.0
1,2012,ФСКН,8.0
2,2013,Роспотребнадзор,7.0
3,2013,ФСКН,7.0
4,2014,Роспотребнадзор,2.0


###### Варианты цветов

In [17]:
colors = ['goldenrod',"red", "green", "blue", "brown", "magenta","grey"]
colors = n_colors('rgb(2, 3, 68)', 'rgb(117, 16, 6)', 7, colortype='rgb')
colors = ['rgb(87, 117, 144)','rgb(67, 170, 139)','rgb(144, 190, 109)','rgb(249, 199, 79)','rgb(248, 150, 30)','rgb(243, 114, 44)','rgb(249, 65, 68)']

##### График

In [22]:
df = dt_year[dt_year['Ведомство'] != 0]

fig = px.bar(df, x="Год", y="Статья", color="Ведомство", text_auto=False, 
             color_discrete_sequence=colors,
             labels={'Статья':'Количество страниц'})

#fig.update_layout(xaxis_range=[2012,2022])
#fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})

fig.update_layout(barmode='relative', title_text='Страницы Википедии, запрещённые в России')
fig.update_xaxes(dtick=1)
fig.show()

#### Сохраняем

In [19]:
plt.io.write_image(fig, "wiki_ban_fig.svg")